In [20]:
import numpy as np
from collections import Counter

def entropy(y):
  split = np.bincount(y)
  ps = split/len(y)
  return -np.sum([p*np.log2(p) for p in ps if p>0])

class Node:

  def __init__(self, feature=None, threshold=None, left=None, right=None, *, value=None):
    self.feature = feature
    self.threshold = threshold
    self.left = left
    self.right = right
    self.value = value

  def is_leaf(self):
    return self.value is not None

class DecisionTree:
  def __init__(self, max_depth= 100, min_samples_split = 2, n_feats= None):
    self.max_depth = max_depth
    self.min_samples_split = min_samples_split
    self.n_feats = n_feats
    self.root = None

  def fit(self, X, y):
    #grow tree
    self.n_feats = X.shape[1] if not self.n_feats else min(X.shape[1], self.n_feats)
    self.root = self._grow_tree(X, y)

  def _grow_tree(self, X, y, depth=0):
    n_samples, n_features = X.shape
    n_labels = len(np.unique(y))


    if (depth> self.max_depth or
        n_samples< self.min_samples_split or
        n_labels == 1):
      leaf_value = self._most_common_label(y)
      return Node(value= leaf_value)

    feat_idxs = np.random.choice(n_features, self.n_feats, replace=False)

    best_feat, best_thresh = self._best_criteria(X, y, feat_idxs)
    left_idxs, right_idxs = self._split(X[:,best_feat], best_thresh)
    left = self._grow_tree(X[left_idxs,:], y[left_idxs], depth+1)
    right = self._grow_tree(X[right_idxs,:], y[right_idxs], depth+1)
    return Node(best_feat, best_thresh, left, right)


  def _best_criteria(self, X, y, feat_idxs):
    best_gain = -1
    split_idx , split_thresh = None, None

    for feat_idx in feat_idxs:
      thresholds = np.unique(X[:,feat_idx])
      for threshold in thresholds:
        gain = self._information_gain(y, X[:, feat_idx], threshold)
        if gain > best_gain:
          best_gain = gain
          split_idx, split_thresh = feat_idx, threshold
    return split_idx, split_thresh

  def _information_gain(self, y, X_column, thresh):
    parent_entropy = entropy(y)

    left_idxs, right_idxs = self._split(X_column, thresh)

    if len(left_idxs)==0 or len(right_idxs)==0:
      return 0

    n_l , n_r = len(left_idxs), len(right_idxs)
    e_l, e_r = entropy(y[left_idxs]), entropy(y[right_idxs])

    child_entropy = e_l*n_l/len(y) + e_r*n_r/len(y)

    ig = parent_entropy - child_entropy
    return ig

  def _split(self, X_column, thresh):
    left_idxs = np.argwhere(X_column<= thresh).flatten()
    right_idxs = np.argwhere(X_column> thresh).flatten()

    return left_idxs, right_idxs

  def _most_common_label(self, y):
    counter = Counter(y)
    return counter.most_common(1)[0][0]

  def predict(self, X):
    return np.array([self._traverse_tree(x, self.root) for x in X])

  def _traverse_tree(self, x, node):
    if node.is_leaf():
      return node.value

    if x[node.feature] <= node.threshold:
      return self._traverse_tree(x, node.left)
    return self._traverse_tree(x, node.right)    




In [52]:
def bootstrap_sample(X, y):
  n_samples = X.shape[0]
  idxs = np.random.choice(n_samples, size= n_samples, replace=True)
  return X[idxs], y[idxs]

def most_common_label(y):
  counter = Counter(y)
  return counter.most_common(1)[0][0]

class RandomForest:

  def __init__(self, n_trees=100, min_samples_split =2, max_depth=100, n_feats=None):
    self.n_trees = n_trees
    self.min_samples_split = min_samples_split
    self.max_depth = max_depth
    self.n_feats = n_feats
    self.trees= []

  def fit(self, X, y):
    self.trees=[]
    for _ in range(self.n_trees):
      tree = DecisionTree(min_samples_split=self.min_samples_split, max_depth=self.max_depth, n_feats=self.n_feats)
      X_sample, y_sample = bootstrap_sample(X, y)
      tree.fit(X_sample, y_sample)
      self.trees.append(tree)

  def predict(self, X):
    tree_preds = np.array([tree.predict(X) for tree in self.trees])
    tree_preds = np.swapaxes(tree_preds, 0, 1)
    y_pred = np.array([most_common_label(tree_pred) for tree_pred in tree_preds])
    return np.array(y_pred)


